In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, balanced_accuracy_score
from imblearn.metrics import classification_report_imbalanced
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

import numpy as np
import re
import plotly as pl 
import matplotlib.pyplot as plt
# Import db related librarys
import psycopg2
import sys
# import boto3
import os
import sqlalchemy
import json
import csv
from getpass import getpass


In [2]:
# Configure connection parameters to AWS RDS
def connect_to_db():
    
    connection = None
    #enter_password = getpass('Enter database password')
    
    try:
        print('Connecting to the PostgreSQL database...')
        connection = psycopg2.connect(
            host = "database-chocolate.cafzzay3t2tr.us-east-2.rds.amazonaws.com",
            port = 5432,
            user = 'postgres',
            password = getpass('Enter database password'),
            database = 'postgres'
            )
        print('Connection successful')
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    return connection


In [3]:
# Call the connect_to_db to connect to database
connection = connect_to_db()

Connecting to the PostgreSQL database...
Enter database password········
Connection successful


In [4]:
# Create cursor to perform database operations
cursor = connection.cursor()

# If curser object = 0, then connection was successfully established
cursor

<cursor object at 0x7f810ca6d650; closed: 0>

In [5]:
# Query the location_table from AWS database and store in dataframe - verify there is data in the table.
sql = """
SELECT * FROM location_table
"""

location_table_from_db = pd.read_sql(sql, con=connection)

In [6]:
# Display the location_table queried from database
location_table_from_db

,country_code,latitude,longitude,broad_bean_origin_country
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla
...,...,...,...,...
242,ZM,-13.133897,27.849332,Zambia
243,ZW,-19.015438,29.154857,Zimbabwe
244,HI,19.898682,-155.665857,Hawaii
245,ZZ,0.000000,0.000000,Unknown


In [7]:
# Query the clean_flavours_table from AWS database and store in dataframe.
sql = """
SELECT * FROM clean_flavors_table
"""

clean_flavors_from_db = pd.read_sql(sql, con=connection)

In [8]:
clean_flavors_from_db

,company,bean_origin_or_bar_name,REF,review_date,cocoa_percent,company_location,rating,bean_type,broad_bean_origin_country,ingredients,most_memorable_characteristics,continent
0,A. Morin,Agua Grande,1876,2016,63.0,France,3.75,missing,Sao Tome & Principe,"4- B,S,C,L","sweet, chocolatey, vegetal",Africa
1,A. Morin,Kpime,1676,2015,70.0,France,2.75,missing,Togo,"4- B,S,C,L","burnt wood, earthy, choco",Africa
2,A. Morin,Atsane,1676,2015,70.0,France,3.00,missing,Togo,"4- B,S,C,L","roasty, acidic, nutty",Africa
3,A. Morin,Akata,1680,2015,70.0,France,3.50,missing,Togo,"4- B,S,C,L","mild profile, chocolaty, spice",Africa
4,A. Morin,Quilla,1704,2015,70.0,France,3.50,missing,Peru,"4- B,S,C,L","grainy texture, cocoa, sweet",South America
...,...,...,...,...,...,...,...,...,...,...,...,...
1955,Zotter,Peru,647,2011,70.0,Austria,3.75,missing,Peru,"4- B,Sw,C,Sa","creamy, fatty, mild nutty",South America
1956,Zotter,Congo,749,2011,65.0,Austria,3.00,Forastero,Republic of Congo,"4- B,Sw,C,Sa","dairy, salt, caramel",Africa
1957,Zotter,Kerala State,749,2011,65.0,Austria,3.50,Forastero,India,"4- B,Sw,C,Sa","creamy, masculine, earthy",Asia
1958,Zotter,Kerala State,781,2011,62.0,Austria,3.25,missing,India,"4- B,Sw,C,Sa","oily, subdued, caramel, salt",Asia


In [9]:
# Join the 2 tables together on broad_bean_origin_country, select the columns you want to view in df

sql = """ SELECT clean_flavors_table .*, location_table.country_code,location_table.longitude, location_table.latitude
FROM clean_flavors_table
LEFT JOIN location_table
ON clean_flavors_table.broad_bean_origin_country = location_table.broad_bean_origin_country;
"""

# Store the joined tables in dataframe
df_chocolate = pd.read_sql(sql, con=connection)

# View the new dataframe combined from two sql tables
df_chocolate.head(10)

,company,bean_origin_or_bar_name,REF,review_date,cocoa_percent,company_location,rating,bean_type,broad_bean_origin_country,ingredients,most_memorable_characteristics,continent,country_code,longitude,latitude
0,A. Morin,Agua Grande,1876,2016,63.0,France,3.75,missing,Sao Tome & Principe,"4- B,S,C,L","sweet, chocolatey, vegetal",Africa,ST,6.613081,0.186360
1,A. Morin,Kpime,1676,2015,70.0,France,2.75,missing,Togo,"4- B,S,C,L","burnt wood, earthy, choco",Africa,TG,0.824782,8.619543
2,A. Morin,Atsane,1676,2015,70.0,France,3.00,missing,Togo,"4- B,S,C,L","roasty, acidic, nutty",Africa,TG,0.824782,8.619543
3,A. Morin,Akata,1680,2015,70.0,France,3.50,missing,Togo,"4- B,S,C,L","mild profile, chocolaty, spice",Africa,TG,0.824782,8.619543
4,A. Morin,Quilla,1704,2015,70.0,France,3.50,missing,Peru,"4- B,S,C,L","grainy texture, cocoa, sweet",South America,PE,-75.015152,-9.189967
5,A. Morin,Carenero,1315,2014,70.0,France,2.75,Criollo,Venezuela,Unknown,missing,South America,VE,-66.589730,6.423750
6,A. Morin,Cuba,1315,2014,70.0,France,3.50,missing,Cuba,"4- B,S,C,L","sliglty dry, papaya",Caribbean,CU,-77.781167,21.521757
7,A. Morin,Sur del Lago,1315,2014,70.0,France,3.50,Criollo,Venezuela,Unknown,missing,South America,VE,-66.589730,6.423750
8,A. Morin,Puerto Cabello,1319,2014,70.0,France,3.75,Criollo,Venezuela,Unknown,missing,South America,VE,-66.589730,6.423750
9,A. Morin,Pablino,1319,2014,70.0,France,4.00,missing,Peru,"4- B,S,C,L","delicate, hazelnut, brownie",South America,PE,-75.015152,-9.189967


In [10]:
# Close connection to RDS
cursor.close()

In [11]:
# Check that connection is closed where object returned = -1
cursor

<cursor object at 0x7f810ca6d650; closed: -1>

# Machine Learning 



In [12]:
def bin_ratings(rating):
    if rating == 5.75: return 1
    if rating == 5.50: return 1
    if rating == 5.25: return 1
    if rating == 5.00: return 1
    
    if rating == 4.75: return 1
    if rating == 4.50: return 1
    if rating == 4.25: return 1
    if rating == 4.00: return 1
    
    if rating == 3.75: return 1
    if rating == 3.50: return 0
    if rating == 3.25: return 0
    if rating == 3.00: return 0

    if rating == 2.75: return 0
    if rating == 2.50: return 0
    if rating == 2.25: return 0
    if rating == 2.00: return 0
    
    if rating == 1.75: return 0
    if rating == 1.50: return 0
    if rating == 1.25: return 0
    if rating == 1.00: return 0
    
    #print( f"error: rating={rating} type={type(rating)}" )
    return "2"

In [13]:
df_chocolate['rating'] = df_chocolate['rating'].apply(bin_ratings)


In [14]:
df_chocolate["rating"].value_counts()

0    1634
1     326
Name: rating, dtype: int64

In [15]:
def to_string(value):
    other = f"_{value}_"
    return other

In [16]:
def reduce_count_vals(df, colname, threshold):
    counts = df[colname].value_counts()
    replace_list = list(counts[counts < threshold].index)

    # Replace in dataframe
    for item in replace_list:
       df[colname] = df[colname].replace(item,"Other")

In [17]:
df_chocolate['review_date']= df_chocolate['review_date'].apply(to_string)
reduce_count_vals(df_chocolate, 'review_date', 100)

## Machine Learning!

In [18]:
# Drop rows where review_date= 2009 or 2010
# indexNames = df_chocolate[(df_chocolate["review_date"]=='_2010_') | (df_chocolate["review_date"]=='_2009_')].index
# df_chocolate.drop(indexNames, inplace=True)
# df_chocolate.head()

In [19]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df_chocolate= df_chocolate.drop(columns=["company_location","REF","bean_origin_or_bar_name","most_memorable_characteristics","latitude","longitude","country_code"], axis=1)
df_chocolate.head()

,company,review_date,cocoa_percent,rating,bean_type,broad_bean_origin_country,ingredients,continent
0,A. Morin,_2016_,63.0,1,missing,Sao Tome & Principe,"4- B,S,C,L",Africa
1,A. Morin,_2015_,70.0,0,missing,Togo,"4- B,S,C,L",Africa
2,A. Morin,_2015_,70.0,0,missing,Togo,"4- B,S,C,L",Africa
3,A. Morin,_2015_,70.0,0,missing,Togo,"4- B,S,C,L",Africa
4,A. Morin,_2015_,70.0,0,missing,Peru,"4- B,S,C,L",South America


In [20]:
df_chocolate.describe()

,cocoa_percent,rating
count,1960.000000,1960.000000
mean,71.760204,0.166327
std,6.657419,0.372469
min,42.000000,0.000000
25%,70.000000,0.000000
50%,70.000000,0.000000
75%,75.000000,0.000000
max,100.000000,1.000000


In [21]:
# Generate our categorical variable lists
chocolate_cat = df_chocolate.dtypes[df_chocolate.dtypes == 'object'].index.tolist()
chocolate_cat

['company',
 'review_date',
 'bean_type',
 'broad_bean_origin_country',
 'ingredients',
 'continent']

In [22]:


# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df_chocolate[chocolate_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(chocolate_cat)
encode_df.head()



,company_A. Morin,company_AMMA,company_Acalli,company_Adi,company_Aequare (Gianduja),company_Ah Cacao,company_Akesson's (Pralus),company_Alain Ducasse,company_Alexandre,company_Altus aka Cao Artisan,...,ingredients_Unknown,continent_Africa,continent_Asia,continent_Caribbean,continent_Central America,continent_North America,continent_Oceania,continent_Other,continent_South America,continent_Unknown
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [23]:
# Merge one-hot encoded features and drop the originals
df_chocolate = df_chocolate.merge(encode_df, left_index=True, right_index=True).drop(chocolate_cat, 1)
df_chocolate.head()

,cocoa_percent,rating,company_A. Morin,company_AMMA,company_Acalli,company_Adi,company_Aequare (Gianduja),company_Ah Cacao,company_Akesson's (Pralus),company_Alain Ducasse,...,ingredients_Unknown,continent_Africa,continent_Asia,continent_Caribbean,continent_Central America,continent_North America,continent_Oceania,continent_Other,continent_South America,continent_Unknown
0,63.0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,70.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,70.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,70.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,70.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [24]:
# Split our preprocessed data into our features and target arrays
y = df_chocolate["rating"]
X = df_chocolate.drop(["rating"],1) 

# Split the preprocessed data into a training and testing dataset
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

X_train, X_test, y_train, y_test = train_test_split(X,
   y,test_size= 0.2,train_size=0.8 , random_state=42, stratify=y)

In [25]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [26]:
len(X_train_scaled[0])

523

In [27]:
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=42)
X_resampled, y_resampled = smote_enn.fit_resample(X_train_scaled, y_train)


# Logistic Regression

In [177]:
classifier = LogisticRegression(solver='lbfgs',
   max_iter=400,
   random_state=28)

In [178]:
classifier.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=400, random_state=28)

In [179]:
y_pred = classifier.predict(X_test_scaled)

In [180]:
print(accuracy_score(y_test, y_pred))

0.8163265306122449


In [181]:
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual average chocolate", "Actual high rated chocolate"], columns=["Predicted average rated chocolate", "Predicted high rated chocolate"])
cm_df

,Predicted average rated chocolate,Predicted high rated chocolate
Actual average chocolate,294,33
Actual high rated chocolate,39,26


In [182]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.90      0.89       327
           1       0.44      0.40      0.42        65

    accuracy                           0.82       392
   macro avg       0.66      0.65      0.66       392
weighted avg       0.81      0.82      0.81       392



In [131]:
# Calculate feature importance in the Logistic Regression.
importances = classifier.coef_[0]

In [132]:
classifier.coef_

array([[-4.07051009e-01,  2.72719507e-01,  5.15738708e-01,
         2.74873784e-01, -3.15597722e-02, -8.83038816e-03,
        -4.88780263e-02,  1.76184296e-01, -1.25105758e-01,
        -9.13414112e-02, -2.80484796e-01,  4.99960576e-01,
        -1.11183989e-01, -9.88773351e-02, -2.10167976e-01,
         7.12003295e-01, -9.77415134e-02, -7.72439965e-02,
        -1.84693701e-01,  2.35249845e-01,  3.16320923e-01,
        -6.23543080e-02,  2.08415867e-01, -1.51690021e-01,
        -1.17966895e-01,  1.35504104e-01,  0.00000000e+00,
         7.24853662e-02, -4.24899605e-02, -1.51791974e-01,
        -9.69967906e-02, -1.60086644e-01, -1.74394090e-01,
         2.23785423e-01,  0.00000000e+00, -1.30849932e-01,
        -1.41951724e-01,  1.27841956e-01, -1.03545311e-01,
        -7.77531011e-02, -2.54767649e-03,  5.98376498e-02,
         4.88382211e-01, -4.42336292e-02,  0.00000000e+00,
         1.59708588e-01, -2.68597710e-01,  0.00000000e+00,
        -1.51989747e-01, -9.75254022e-02, -8.48186801e-0

In [133]:
# We can sort the features by their importance.
sorted(zip(classifier.coef_[0], X.columns), reverse=True)

[(0.7120032947337365, 'company_Amedei'),
 (0.6518203148890402, 'company_Michel Cluizel'),
 (0.5157387080648234, 'company_AMMA'),
 (0.4999605758154924, 'company_Amano'),
 (0.4990202344210256, 'company_Soma'),
 (0.4887490462686628, 'company_Heirloom Cacao Preservation (Zokoko)'),
 (0.4883822109867853, 'company_Bonnat'),
 (0.43044654947114047, 'company_Zotter'),
 (0.3764598224767058, 'broad_bean_origin_country_Madagascar'),
 (0.36573949160475183, 'company_Scharffen Berger'),
 (0.3635890651422448, 'continent_South America'),
 (0.3481028636287146, 'company_Danta'),
 (0.3379039298675012, 'company_Domori'),
 (0.3311061952663341, "company_Duffy's"),
 (0.3291784934621917, 'company_Debauve & Gallais (Michel Cluizel)'),
 (0.3269316771875179, 'company_Valrhona'),
 (0.32628382525553973, 'broad_bean_origin_country_Ghana'),
 (0.32574580883011167, 'company_Pierre Marcolini'),
 (0.3198379304708268, 'company_Rogue'),
 (0.31724216344398337, 'company_Idilio (Felchlin)'),
 (0.3163209231816928, 'company_Art

# Random Forest

In [231]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=500, random_state=28, criterion='entropy', min_samples_split=2,
                                  min_samples_leaf=1,max_features=100)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")



 Random forest predictive accuracy: 0.842


In [232]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
accuracy_score(y_test, y_pred)

0.8418367346938775

In [233]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual average chocolate", "Actual high rated chocolate"], columns=["Predicted average rated chocolate", "Predicted high rated chocolate"])
cm_df

,Predicted average rated chocolate,Predicted high rated chocolate
Actual average chocolate,305,22
Actual high rated chocolate,40,25


In [234]:
# Print the imbalanced classification report

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.88      0.93      0.38      0.91      0.60      0.38       327
          1       0.53      0.38      0.93      0.45      0.60      0.34        65

avg / total       0.83      0.84      0.48      0.83      0.60      0.37       392



In [209]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_



In [210]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.14719918917077107, 'company_Soma'),
 (0.1113160326441536, 'cocoa_percent'),
 (0.04792197774525688, 'continent_South America'),
 (0.046909438796505845, 'bean_type_missing'),
 (0.04375388098649659, 'broad_bean_origin_country_Venezuela'),
 (0.020314921066452402, 'company_Bonnat'),
 (0.019849494115366, 'bean_type_Criollo'),
 (0.0194704036748338, 'review_date__2016_'),
 (0.01682509255003594, 'ingredients_3- B,S,C'),
 (0.016672610883710182, 'company_Fresco'),
 (0.016291031724967164, 'ingredients_Unknown'),
 (0.015845317454221977, 'review_date__2009_'),
 (0.014277103996189458, 'review_date_Other'),
 (0.013095012521376456, 'bean_type_Trinitario'),
 (0.012588314488161704, 'company_AMMA'),
 (0.012212962143353934, 'broad_bean_origin_country_Peru'),
 (0.011948988653098267, 'broad_bean_origin_country_Ecuador'),
 (0.01124969356423359, 'company_Arete'),
 (0.011241036604386193, 'company_Madecasse (Cinagra)'),
 (0.010972502985528881, 'review_date__2015_'),
 (0.009764299197623724, 'review_date__2012

# Balanced RandomForest Classifier

In [56]:
from imblearn.ensemble import BalancedRandomForestClassifier
brf_model= BalancedRandomForestClassifier(n_estimators=500, random_state=42, criterion='entropy', min_samples_leaf=1, max_features=30)
brf_model.fit(X_train_scaled, y_train)
y_pred_brf = brf_model.predict(X_test_scaled)

In [57]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test,y_pred_brf)

0.7134556574923547

In [58]:
# Display the confusion matrix
matrix= confusion_matrix(y_test, y_pred_brf)
print(matrix)

[[205 122]
 [ 13  52]]


In [59]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_brf))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.94      0.63      0.80      0.75      0.71      0.49       327
          1       0.30      0.80      0.63      0.44      0.71      0.51        65

avg / total       0.83      0.66      0.77      0.70      0.71      0.50       392



In [60]:
importances = brf_model.feature_importances_
importances
from operator import itemgetter
ordered_importances= sorted(zip(X.columns,importances), key=itemgetter(1), reverse=True)
ordered_importances

[('cocoa_percent', 0.10625026180368081),
 ('company_Soma', 0.017263593898580294),
 ('bean_type_missing', 0.017076335026303197),
 ('ingredients_3- B,S,C', 0.01635672085815606),
 ('ingredients_2- B,S', 0.015309044630044464),
 ('review_date__2015_', 0.0147159404323561),
 ('ingredients_Unknown', 0.013357531612318323),
 ('review_date__2014_', 0.013267569166245089),
 ('review_date__2016_', 0.013128614932686656),
 ('broad_bean_origin_country_Venezuela', 0.012926699815736924),
 ('bean_type_Trinitario', 0.012903579921003763),
 ('bean_type_Criollo', 0.01258706381094171),
 ('review_date__2012_', 0.012410348328120836),
 ('review_date__2013_', 0.012247404903270668),
 ('continent_South America', 0.012089794132481381),
 ('review_date__2011_', 0.011572619414724757),
 ('review_date_Other', 0.011192577557542901),
 ('ingredients_4- B,S,C,L', 0.01103140988697001),
 ('review_date__2009_', 0.011026903260207143),
 ('review_date__2010_', 0.010580233075815964),
 ('broad_bean_origin_country_Peru', 0.01031052368

# Support Vector Machine

In [50]:
# Create the SVM model
svm = SVC(kernel='rbf')

# Train the model
svm.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = svm.predict(X_test_scaled)
print(f" SVM model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 SVM model accuracy: 0.837


In [51]:
# Calculated the balanced accuracy score
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.8367346938775511

In [52]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual average chocolate", "Actual high rated chocolate"], columns=["Predicted average rated chocolate", "Predicted high rated chocolate"])
cm_df

,Predicted average rated chocolate,Predicted high rated chocolate
Actual average chocolate,316,11
Actual high rated chocolate,53,12


In [64]:
# Print the classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.97      0.91       327
           1       0.52      0.18      0.27        65

    accuracy                           0.84       392
   macro avg       0.69      0.58      0.59       392
weighted avg       0.80      0.84      0.80       392



# GradientBoost

In [65]:
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    classifier = GradientBoostingClassifier(n_estimators=20,
    learning_rate=learning_rate,
    max_features=5,
    max_depth=3,
    random_state=42)
    classifier.fit(X_train_scaled, y_train)
    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(
           classifier.score(
               X_train_scaled,
               y_train)))
    print("Accuracy score (validation): {0:.3f}".format(
           classifier.score(
               X_test_scaled,
               y_test)))

Learning rate:  0.05
Accuracy score (training): 0.834
Accuracy score (validation): 0.834
Learning rate:  0.1
Accuracy score (training): 0.834
Accuracy score (validation): 0.834
Learning rate:  0.25
Accuracy score (training): 0.839
Accuracy score (validation): 0.839
Learning rate:  0.5
Accuracy score (training): 0.858
Accuracy score (validation): 0.834
Learning rate:  0.75
Accuracy score (training): 0.856
Accuracy score (validation): 0.819
Learning rate:  1
Accuracy score (training): 0.857
Accuracy score (validation): 0.814


In [66]:
classifier = GradientBoostingClassifier(n_estimators=20,
   learning_rate=0.75, max_features=5, max_depth=3, random_state=0)

classifier.fit(X_train_scaled, y_train)
predictions = classifier.predict(X_test_scaled)

In [67]:
acc_score = accuracy_score(y_test, predictions)
print(f"Accuracy Score : {acc_score}")

Accuracy Score : 0.8392857142857143


In [68]:
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
   cm, index=["Actual 0", "Actual 1"],
   columns=["Predicted 0", "Predicted 1"]
)
display(cm_df)
print("Classification Report")
print(classification_report(y_test, predictions))

,Predicted 0,Predicted 1
Actual 0,317,10
Actual 1,53,12


Classification Report
              precision    recall  f1-score   support

           0       0.86      0.97      0.91       327
           1       0.55      0.18      0.28        65

    accuracy                           0.84       392
   macro avg       0.70      0.58      0.59       392
weighted avg       0.81      0.84      0.80       392



# Deep Learning Model

In [69]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  256
hidden_nodes_layer2 = 128
hidden_nodes_layer3 = 64
hidden_nodes_layer4 = 32
hidden_nodes_layer5 = 16
hidden_nodes_layer6 = 8
hidden_nodes_layer7 = 4


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer1, input_dim = number_input_features, activation = 'relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer2, activation = 'relu'))

# other hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer3, activation = 'relu'))
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer4, activation = 'relu'))
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer5, activation = 'relu'))
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer6, activation = 'relu'))
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer7, activation = 'relu'))


# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               134144    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 3

In [70]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [71]:
# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics= ['accuracy'])

In [72]:
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

In [73]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs = 100, callbacks=[cp_callback])

Epoch 1/100
49/49 [==============================] - 1s 2ms/step - loss: 0.4604 - accuracy: 0.8323
Epoch 2/100
49/49 [==============================] - 0s 2ms/step - loss: 0.3463 - accuracy: 0.8323
Epoch 3/100
49/49 [==============================] - 0s 2ms/step - loss: 0.2726 - accuracy: 0.8571
Epoch 4/100
49/49 [==============================] - 0s 2ms/step - loss: 0.2262 - accuracy: 0.8960
Epoch 5/100
49/49 [==============================] - 0s 2ms/step - loss: 0.2013 - accuracy: 0.9075

Epoch 00005: saving model to checkpoints/weights.05.hdf5
Epoch 6/100
49/49 [==============================] - 0s 2ms/step - loss: 0.1640 - accuracy: 0.9286
Epoch 7/100
49/49 [==============================] - 0s 2ms/step - loss: 0.1460 - accuracy: 0.9362
Epoch 8/100
49/49 [==============================] - 0s 2ms/step - loss: 0.1284 - accuracy: 0.9426
Epoch 9/100
49/49 [==============================] - 0s 2ms/step - loss: 0.1234 - accuracy: 0.9496
Epoch 10/100
49/49 [==============================]

In [74]:


# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")



13/13 - 0s - loss: 2.8195 - accuracy: 0.7628
Loss: 2.819486379623413, Accuracy: 0.7627550959587097


In [75]:
nn.save("Chocolate_Ratings_ML.h5")